Imports

In [1]:
import f90nml
import numpy as np
from pint import UnitRegistry; AssignQuantity = UnitRegistry().Quantity
from QLCstuff2 import getNQLL
import os
import reference_solution as refsol
from scipy.fft import rfft
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import icepinn as ip

torch.set_default_dtype(torch.float64)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

device = ip.get_device()

cuda
cuda


In [2]:
# Read in GI parameters
inputfile = "GI parameters - Reference limit cycle (for testing).nml"
GI=f90nml.read(inputfile)['GI']
nx_crystal = GI['nx_crystal']
L = GI['L']
NBAR = GI['Nbar']
NSTAR = GI['Nstar']

# Define t range
RUNTIME = 5
NUM_T_STEPS = RUNTIME + 1

# Define initial conditions
Ntot_init_1D = np.ones(nx_crystal)
Nqll_init_1D = getNQLL(Ntot_init_1D,NSTAR,NBAR)

# Define x, t pairs for training
X_QLC = np.linspace(-L,L,nx_crystal)
t_points = np.linspace(0, RUNTIME, NUM_T_STEPS)
x, t = np.meshgrid(X_QLC, t_points)
training_set = torch.tensor(np.column_stack((x.flatten(), t.flatten()))).to(device)

In [3]:
# Define model dimensions; instantiate model
model_dimensions = torch.tensor([8, 80]).to(device)
model = ip.IcePINN(num_hidden_layers=model_dimensions[0], hidden_layer_size=model_dimensions[0]).to(device)

# Attach model dimensions so they can be saved and loaded
model.register_buffer('dimensions', model_dimensions)

# Initialize model weights with HE initialization
def init_HE(m):
		if type(m) == nn.Linear:
			nn.init.kaiming_normal_(m.weight)
model.apply(init_HE)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

Train model

In [4]:
MODEL_NAME = "TestPINN"
PATH = './models/'+MODEL_NAME

ip.train_IcePINN(model, optimizer, training_set, epochs=50_000, save_path=PATH, print_every=1_000, print_gradients=False)

Commencing PINN training for 50000 epochs.
Loss at epoch [1000/50000]: Ntot = 13.757093462720707, Nqll = 8.27980181594701.
Loss at epoch [2000/50000]: Ntot = 5.603213241141843, Nqll = 2.683554053345563.
Loss at epoch [3000/50000]: Ntot = 10.168766458773534, Nqll = 6.1842247754923925.
Loss at epoch [4000/50000]: Ntot = 91.34929496192608, Nqll = 23.066536656874767.
Loss at epoch [5000/50000]: Ntot = 34.520173343113655, Nqll = 8.443160287299674.
Loss at epoch [6000/50000]: Ntot = 18.608560311543265, Nqll = 4.4535291684149545.
Loss at epoch [7000/50000]: Ntot = 3.850059813321768, Nqll = 0.8536307604814424.
Loss at epoch [8000/50000]: Ntot = 28.712023556722283, Nqll = 7.214190461881078.
Loss at epoch [9000/50000]: Ntot = 21.47437801302649, Nqll = 4.953943879977403.
Loss at epoch [10000/50000]: Ntot = 19.614871288081186, Nqll = 4.512732414437673.
Loss at epoch [11000/50000]: Ntot = 8.673752031317893, Nqll = 2.0023617263322873.
Loss at epoch [12000/50000]: Ntot = 7.016224635958822, Nqll = 1.6

Save model

In [5]:



# torch.save(model.state_dict(), PATH'/'+MODEL_NAME+'.pth')
# torch.save(optimizer.state_dict(), PATH'/'+MODEL_NAME+'.pth')

Evaluate Model (visually?)

In [6]:
# loaded_model = IcePINN()
# loaded_model.load_state_dict(torch.load(PATH)) # it takes the loaded dictionary, not the path file itself
# loaded_model.to(device)
# loaded_model.eval()